In [0]:
# importações

from pyspark.sql.functions import avg, count, col, coalesce, lit


In [0]:
# Leitura do DataFrame
avaliacoes_df = spark.table("ecommerce.silver.avaliacoes")
produtos_df = spark.table("ecommerce.silver.produtos")
categorias_df = spark.table("ecommerce.silver.categorias")

In [0]:
# Agregações das avaliações por produto

avaliacoes_agg_df = (
    avaliacoes_df
        .groupBy("produto_id")
        .agg(
            avg("nota").alias("media_avaliacao"),
            count("avaliacao_id").alias("total_avaliacoes")
        )
)

# Aliases

pr = produtos_df.alias("pr")
cat = categorias_df.alias("cat")
av = avaliacoes_agg_df.alias("av")

#Criando a silver produtos completos

produtos_completos_df = (
    pr
    # Categoria
    .join(cat, pr.categoria_id == cat.categoria_id, "left")
    # Avaliações agregadas
    .join(av, pr.produto_id == av.produto_id, "left")
    .select(
        col("pr.produto_id"),
        col("pr.nome").alias("produto_nome"),
        col("pr.marca"),
        col("pr.preco"),
        col("cat.nome").alias("categoria_nome"),
        coalesce(col("av.media_avaliacao"), lit(0)).alias("media_avaliacao"),
        coalesce(col("av.total_avaliacoes"), lit(0)).alias("total_avaliacoes")
    )
)

In [0]:
#Salvando DataFrame na camada silver
produtos_completos_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("ecommerce.silver.produtos_completos")